# Práctica 3: Búsqueda Local

Esta práctica propone implementar algunos de los algoritmos de **búsqueda local** vistos en clase y usarlos para resolver los problemas de las **8 reinas** (8-queens) y el **juego del 8** (8-puzzle). 

Para agilizar el trabajo, en el archivo `problem.py` se provee la clase `SearchProblem` que implementa un problema general de búsqueda. Esta clase posee los siguientes métodos:

* `estado_inicial(self) -> Any`: Devuelve el estado inicial.

* `acciones(self, estado: Any) -> List[Any]`: Devuelve la lista de acciones aplicables en el estado dado.

* `resultado(self, estado: Any, accion: Any) -> Any`: Devuelve el estado resultado de aplicar acción sobre estado.

* `valor_objetivo(self, estado: Any) -> float`: Devuelve el valor objetivo asociado al estado.

* `estado_aleatorio(self) -> Any`: Genera y devuelve un estado aleatorio del problema.

Además, de ella heredan las clases `EightPuzzleProblem` y `EightQueensProblem` que implementan los métodos específicos para el 8-puzzle y 8-queens, respectivamente. Sus **formulaciones de estados completas** siguen las propuestas en clase. Puede consultar estas definiciones en `problem.py`, aunque no es estríctamente necesario para la realización de esta práctica.

Comenzamos entonces importando estas clases.

In [2]:
from problem import SearchProblem, EightPuzzleProblem, EightQueensProblem

Además, importamos algunos paquetes necesarios.

In [1]:
from typing import Any, Tuple, Callable
import random

## Descenso de colinas

En esta sección se debe realizar una implementación del algoritmo de descenso de colinas, respetando la plantilla de abajo. Tener en cuenta que, a diferencia de la ascensión de colinas vista en clase, acá se pide un descenso de colinas, es decir, hay que moverse al sucesor que más reduzca el valor objetivo. Esto se debe a que los dos problemas que se trabajan en esta práctica son de minimización y resulta más claro invertir el ascenso en vez de trabajar con el opuesto de la función objetivo para resolverlo como un problema de maximización.

In [3]:
def descenso_colinas(problem: SearchProblem) -> Tuple[Any, float, int]:
    """Resuelve un problema de optimizacion con descenso de colinas.

    Argumentos:
    ==========
    problem: SearchProblem
        un problema de optimizacion

    Retorna:
    =======
    Tuple[Any, float, int]
        una tupla con estado final, el valor objetivo del estado final y el número de iteraciones realizadas

    Raises:
    =======
    ValueError
        si no hay sucesores disponibles desde el estado actual
    
    """
    # TODO: COMPLETAR
    raise NotImplementedError

A continuación se proponen algunos casos de prueba para testear la implementación de `descenso_colinas`.

In [5]:
# Caso de prueba: estado inicial ya minimo global
estado1 =  [5, 3, 1, 7, 2, 8, 6, 4]
reinas1 = EightQueensProblem(estado1)
sol1 = descenso_colinas(reinas1)
assert sol1[0] == estado1 and sol1[1] == 0 and sol1[2] == 0, "Fallo en caso de prueba 1"

In [6]:
# Caso de prueba: estado inicial a un paso del minimo global
estado2 =  [1, 3, 1, 7, 2, 8, 6, 4]
reinas2 = EightQueensProblem(estado2)
sol2 = descenso_colinas(reinas2)
assert sol2[0] == estado1 and sol2[1] == 0 and sol2[2] == 1, "Fallo en caso de prueba 2"

In [7]:
# Caso de prueba: estado inicial es minimo local
estado3 =  [8, 4, 1, 7, 2, 6, 3, 5]
reinas3 = EightQueensProblem(estado3)
sol3 = descenso_colinas(reinas3)
assert sol3[0] == estado3 and sol3[1] == 1 and sol3[2] == 0, "Fallo en caso de prueba 3"

In [8]:
# Tests aleatorios (sanity check)
# Verifica que descenso_colinas nunca aumenta el valor objetivo inicial.

n_pruebas = 20
errores = 0

for i in range(n_pruebas):
    # Generar un estado aleatorio de 8 reinas
    problem = EightQueensProblem()
    estado_ini = problem.estado_inicial()
    valor_ini = problem.valor_objetivo(estado_ini)

    # Ejecutar descenso de colinas
    try:
        estado_fin, valor_fin, _ = descenso_colinas(problem)
    except ValueError:
        # Si no hay sucesores disponibles, simplemente continuar
        continue

    # Verificar condición de no aumento
    if valor_fin > valor_ini:
        print(f"❌ Error en prueba {i+1}: valor inicial {valor_ini}, valor final {valor_fin}")
        errores += 1

if errores == 0:
    print(f"✅ Todas las {n_pruebas} ejecuciones mantuvieron o mejoraron el valor objetivo.")
else:
    print(f"⚠️ {errores} ejecuciones empeoraron el valor objetivo.")

Utilizar la función `simular`, provista en la celda de abajo, para realizar 1000 corridas del descenso de colinas para resolver el 8-queens desde estados iniciales aleatorios. Observar el porcentaje de ejecuciones que acabaron en un mínimo global o local, así como el promedio de iteraciones realizadas. 

* ¿Los resultados obtenidos con la simulación coinciden con los mencionados en teoría?

In [9]:
def simular(nruns: int, 
            problem: SearchProblem, 
            search: Callable[[SearchProblem], Tuple[Any, float, int]]) -> None:
    """Simula nruns ejecuciones de search desde estados iniciales aleatorios de problem.

    Imprime por pantalla el porcentaje de ejecuciones que acabaron en un mínimo global o local,
    así como el promedio de iteraciones realizadas. 

    Advertencia: se asume que las soluciones con valor objetivo 0 son mínimos globales.

    Argumentos:
    ==========
    nruns: int
        número de ejecuciones a simular
    problem: SearchProblem
        un problema de optimizacion
    search: Callable[[SearchProblem], Tuple[Any, float, int]]
        una función de búsqueda local que resuelve problem
    """
    nglobal = 0
    nlocal = 0
    total_iters_local = 0
    total_iters_global = 0
    for _ in range(nruns):
        problem._initial = problem.estado_aleatorio()
        try:
            _, val, iters = search(problem)
        except ValueError:
            continue  # si no hay sucesores, saltar ejecución
        if val == 0: # mínimo global
            nglobal += 1
            total_iters_global += iters
        else:  # mínimo local
            nlocal += 1
            total_iters_local += iters
    total = nglobal + nlocal
    print(f"Corridas: {total}")
    if total > 0:
        print(f"Mínimos globales: {nglobal/total*100:.1f}%")
        print(f"Mínimos locales:  {nlocal/total*100:.1f}%")
    if nglobal > 0:
        print(f"Promedio iteraciones (globales): {total_iters_global/nglobal:.2f}")
    if nlocal > 0:
        print(f"Promedio iteraciones (locales):  {total_iters_local/nlocal:.2f}")

In [10]:
simular(1000, EightQueensProblem(), descenso_colinas)

## Movimientos laterales

Ahora queremos incorporar movimientos laterales al descenso de colinas. Utilice el argumento `max_sideways` para limitar el número de movimientos laterales consecutivos. Tenga en cuenta que el contador de movimientos laterales se debe resetear a 0 cada vez que el descenso haga un movimiento que no sea lateral. 

**Importante.** Para mitigar las chances de que los movimientos laterales sigan caminos cíclicos, procurar resolver los empates de forma **aleatoria**. Es decir, si el menor valor objetivo lo alcanzan múltiples sucesores, entonces elegir uno al azar.

In [11]:
def descenso_colinas_pasos_laterales(problem: SearchProblem, max_sideways: int = 100) -> Tuple[Any, float, int]:
    """Resuelve un problema de optimizacion con descenso de colinas con movimientos laterales.

    Argumentos:
    ==========
    problem: SearchProblem
        un problema de optimizacion
    max_sideways: int
        número máximo de movimientos laterales permitidos

    Retorna:
    =======
    Tuple[Any, float, int]
        una tupla con estado final, el valor objetivo del estado final y el número de iteraciones realizadas

    Raises:
    =======
    ValueError
        si no hay sucesores disponibles desde el estado actual
    
    """
    # TODO: COMPLETAR
    raise NotImplementedError

Volver a simular 1000 corridas del descenso de colinas con movimientos latereales para resolver el 8-queens desde estados iniciales aleatorios. 

* ¿Los resultados obtenidos con la simulación coinciden con los mencionados en teoría?

In [13]:
simular(1000, EightQueensProblem(), descenso_colinas_pasos_laterales)

## Limitaciones del descenso de colinas

Simular 1000 corridas del descenso de colinas sin y con movimientos latereales para resolver el **8-puzzle** desde estados iniciales aleatorios. Responder:

* ¿Hay alguna diferencia respecto a los resultados obtenidos para el 8-queens? Especular cuáles pueden ser los motivos.

In [14]:
simular(1000, EightPuzzleProblem(), descenso_colinas)

In [15]:
simular(1000, EightPuzzleProblem(), descenso_colinas_pasos_laterales)

## Búsqueda tabú

En esta sección se debe realizar una implementación del algoritmo de búsqueda tabú, siguiendo la plantilla propuesta en la celda de abajo. 
Considerar las siguientes pautas:

* Moverse siempre al sucesor no tabú con **menor** valor objetivo (a diferencia de la implementación vista en teoría que se movía al de **mayor** valor objetivo). Es decir, la implementación debe ser complatible con problemas de **minimización**.

* Como criterio de parada de la búsqueda tabú usar un número máximo de iteraciones `max_iterations`.

* Para la lista tabú usar un TAD cola, por ejemplo, una `deque` del paquete `collections`.

* Almacenar en la lista tabú los **estados** visitados recientemente (a diferencia de la implementación vista en teoría que almacenaba acciones).

* Limitar la capacidad de la lista tabú según el valor del parámetro `tabu_size`. En caso de que la lista supere esta capacidad, quitar el estado más antiguo.

* Al igual que en el descenso de colinas con movimientos laterales, procurar resolver los empates de forma **aleatoria**, para mitigar las chances de seguir caminos cíclicos. Es decir, si el menor valor objetivo lo alcanzan múltiples sucesores no tabúes, entonces elegir uno al azar.

In [16]:
from collections import deque

In [17]:
def busqueda_tabu(problem: SearchProblem, 
                  max_iterations: int = 500, 
                  tabu_size: int = 5) -> Tuple[Any, float, int]:
    """Resuelve un problema de optimizacion con búsqueda tabú.

    Argumentos:
    ==========
    problem: SearchProblem
        un problema de optimizacion
    max_iterations: int
        número máximo de iteraciones permitidas
    tabu_size: int
        capacidad de la lista tabú

    Retorna:
    =======
    Tuple[Any, float, int]
        una tupla con el mejor estado encontrado, el valor objetivo de ese estado y el número de iteraciones realizadas

    Raises:
    =======
    ValueError
        si no hay sucesores disponibles desde el estado actual
    """
    # TODO: COMPLETAR
    raise NotImplementedError

Simular 1000 corridas de la búsqueda tabú para resolver el 8-puzzle desde estados iniciales aleatorios.

* Experimentar diferentes valores para los parámetros del algoritmo. Decidir la mejor combinación de valores.

* Comparar los resulados obtenidos con la ascensión de colinas sin y con movimientos laterales.

* Concluir cuál algoritmo de búsqueda local es más efectivo para el 8-puzzle y especular cuáles pueden ser los motivos.

In [19]:
simular(1000, EightPuzzleProblem(), busqueda_tabu)